# Pre processing and Tree Segmentation

In [ ]:
from whitebox import WhiteboxTools
import laspy
import pdal
import json
import numpy as np
import sys
from datetime import datetime
import rasterio

from pathlib import Path

# set root directory
BASE_DIR = Path.cwd()

# Falls dein preprocessing-Ordner im Projekt liegt:
WORK_DIR = BASE_DIR / "preprocessing"

# Whitebox initialisieren
wbt = WhiteboxTools()
wbt.set_working_dir(str(WORK_DIR))

print("Working directory:", wbt.work_dir)

# Ground classification with Cloth Simulation Filter (CSF) [Zhang et al., 2016]

In [ ]:
# --- Define folders ---
INPUT_FILE = WORK_DIR / "Wenns_Data" / "PC" / "Wenns_aoi.las"
OUTPUT_DIR = WORK_DIR / "output_lidar"

# --- Output ---
ALL_FILE = OUTPUT_DIR / "las_ground_nonground.las"
GROUND_FILE = OUTPUT_DIR / "ground_csf.las"
NONGROUND_FILE = OUTPUT_DIR / "nonground_csf.las"

In [ ]:
# initialize json pipeline
json_pipeline = {
    "pipeline": [
        {
            "type": "readers.las",
            "filename": str(INPUT_FILE)
        },

        {
            "type": "filters.assign",
            "value": [
            "ReturnNumber = 1 WHERE ReturnNumber < 1",
            "NumberOfReturns = 1 WHERE NumberOfReturns < 1"
            ]
        },

        {
            "type": "filters.outlier"
        },
        {
            "type": "filters.csf",
            "resolution": 0.2,
        },
        # Export all
        {
            "type": "writers.las",
            "filename": str(ALL_FILE),
        },

        # Ground export
        {
            "type": "writers.las",
            "filename": str(GROUND_FILE),
            "where": "Classification == 2"
        },

        # Non-ground export
        {
            "type": "writers.las",
            "filename": str(NONGROUND_FILE),
            "where": "Classification != 2"
        }
    ]
}

json_pipeline = json.dumps(json_pipeline)

pipeline = pdal.Pipeline(json_pipeline)
count = pipeline.execute()

# Create DTM, DSM and CHM

In [ ]:
las_ground_nonground = laspy.read("preprocessing/output_lidar/las_ground_nonground.las")

xmin, xmax = np.min(las_ground_nonground.x), np.max(las_ground_nonground.x)
ymin, ymax = np.min(las_ground_nonground.y), np.max(las_ground_nonground.y)

print(xmin, xmax, ymin, ymax)

## Check for different classifications so they can be excluded in the next step 

In [ ]:
print(np.unique(las_ground_nonground.classification))

## Create DEM (IDW)

In [ ]:
OUTPUT_TIF = WORK_DIR / "output_rast"

INPUT_LAS = OUTPUT_DIR / "las_ground_nonground.las"
DEM_FILE = OUTPUT_TIF / "raw_dem.tif"
DSM_FILE = OUTPUT_TIF / "raw_dsm.tif"
CHM_FILE = OUTPUT_TIF / "raw_chm.tif"

In [ ]:
print("Interpolating DEM...")
wbt.lidar_idw_interpolation(
i=str(INPUT_LAS),
output=str(DEM_FILE),
parameter="elevation",
returns="all",
resolution=0.2,
weight=1.0,
radius=2.5,
exclude_cls='0,1,7'
)

## Create DSM (IDW)

In [ ]:
print("Interpolating DSM...")
wbt.lidar_idw_interpolation(
i=str(INPUT_LAS),
output=str(DSM_FILE),
parameter="elevation",
returns="first",
resolution=0.2,
weight=1.0,
radius=2.5
)

## Create CHM

In [ ]:
print("Creating CHM...")
wbt.subtract(
    input1=str(DSM_FILE),
    input2=str(DEM_FILE),
    output=str(CHM_FILE)
)

# Define CRS for DTM,DSM and CHM and smooth with gaussina filter

In [ ]:
from scipy.ndimage import median_filter 

with rasterio.open(str(DEM_FILE), "r+") as dataset:
    dataset.crs = "EPSG:32632"
    print(dataset.crs)

with rasterio.open(str(DSM_FILE), "r+") as dataset:
    dataset.crs = "EPSG:32632"
    print(dataset.crs)
    

with rasterio.open(str(CHM_FILE), "r+") as dataset:
    dataset.crs = "EPSG:32632"
    print(dataset.crs)

    chm = dataset.read(1, masked=True).astype("float32")
    profile = dataset.profile

    chm_filled = chm.filled(0)
    chm_smoothed = median_filter(chm_filled, size=5)

    # mask no-data
    chm_smoothed[chm.mask] = dataset.nodata

    profile.update(dtype=chm_smoothed.dtype)

    with rasterio.open("chm_filtered.tif", "w", **profile) as dst:
        dst.write(chm_smoothed, 1)


# Normalize Pointcloud

In [ ]:
INPUT_LAS = OUTPUT_DIR / "nonground_csf.las"
NORM_LAS = OUTPUT_DIR / "normalized_pc.las"

In [ ]:
wbt.normalize_lidar(
    i=str(INPUT_LAS),
    output=str(NORM_LAS),
    dtm=str(DEM_FILE)
)

In [ ]:
las_normalized = laspy.read(str(NORM_LAS))
np.min(las_normalized.z)

# Individual Tree Detection

In [ ]:
wbt.individual_tree_detection(
    i=str(NORM_LAS), 
    output="treetops.shp", 
    min_search_radius=1.0, 
    min_height=5.0, 
    max_search_radius="4", 
    max_height="40", 
    only_use_veg=False
)

# Before continuing run treeiso.py in command line on non ground pointcloud to segment individual trees!!!

In [ ]:
INPUT_LAS = OUTPUT_DIR / "nonground_csf_treeiso.laz"
las_trees = laspy.read(str(INPUT_LAS))

print(las_trees.header.vlrs)

# Pycrown updated

In [ ]:
import numpy as np
from numba import njit


# --------------------------------------------------
# 1. Create circular neighbourhood offsets
# --------------------------------------------------
@njit
def create_circular_offsets(radius):
    offsets = []
    r2 = radius * radius
    for dy in range(-radius, radius + 1):
        for dx in range(-radius, radius + 1):
            if dx * dx + dy * dy <= r2:
                offsets.append((dy, dx))
    return offsets


# --------------------------------------------------
# 2. Modern Dalponte CIRC
# --------------------------------------------------
@njit
def crown_dalponte_circ(
    chm,
    tree_coords,     # shape (2, N) -> [x_coords, y_coords]
    th_seed=0.45,
    th_crown=0.55,
    th_tree=2.0,
    max_crown=10
):
    nrows, ncols = chm.shape
    ntrees = tree_coords.shape[1]

    crowns = np.zeros((nrows, ncols), dtype=np.int32)

    # Track crown stats
    npixel = np.ones(ntrees)
    sum_height = np.zeros(ntrees)

    offsets = create_circular_offsets(max_crown)

    # Initialize seeds
    for i in range(ntrees):
        x = tree_coords[0, i]
        y = tree_coords[1, i]
        crowns[y, x] = i + 1
        sum_height[i] = chm[y, x]

    growing = True

    while growing:
        growing = False

        for tidx in range(ntrees):

            seed_x = tree_coords[0, tidx]
            seed_y = tree_coords[1, tidx]
            seed_h = chm[seed_y, seed_x]
            mean_h = sum_height[tidx] / npixel[tidx]

            for dy, dx in offsets:

                ny = seed_y + dy
                nx = seed_x + dx

                # Bounds check
                if ny < 1 or ny >= nrows - 1:
                    continue
                if nx < 1 or nx >= ncols - 1:
                    continue

                if crowns[ny, nx] != 0:
                    continue

                nb_h = chm[ny, nx]

                # Dalponte growing criteria
                if (
                    nb_h > th_tree and
                    nb_h > seed_h * th_seed and
                    nb_h > mean_h * th_crown and
                    nb_h <= seed_h * 1.05 and
                    abs(seed_x - nx) <= max_crown and
                    abs(seed_y - ny) <= max_crown
                ):

                    # 4-connectivity check
                    if (
                        crowns[ny - 1, nx] == tidx + 1 or
                        crowns[ny + 1, nx] == tidx + 1 or
                        crowns[ny, nx - 1] == tidx + 1 or
                        crowns[ny, nx + 1] == tidx + 1
                    ):
                        crowns[ny, nx] = tidx + 1
                        sum_height[tidx] += nb_h
                        npixel[tidx] += 1
                        growing = True

    return crowns

In [ ]:
# Export tree crowns Shapefile

In [ ]:
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape, mapping
import fiona

def export_crowns_polygons(crowns, transform, epsg, out_file):
    """
    Convert crowns raster to polygons and save as Shapefile
    """
    schema = {
        'geometry': 'Polygon',
        'properties': {'DN': 'int'}
    }

    with fiona.open(out_file, 'w', driver='ESRI Shapefile',
                    crs=f"EPSG:{epsg}", schema=schema) as shp:
        for geom, value in shapes(crowns.astype(np.int32), mask=crowns>0,
                                  transform=transform):
            shp.write({
                'geometry': mapping(shape(geom)),
                'properties': {'DN': int(value)}
            })
    print(f"Crowns shapefile exported: {out_file}")

## Use circular dalponte

In [ ]:
## Import treetops

In [ ]:
import geopandas as gpd

# Tree tops Shapefile
tree_shp = "tree_tops.shp"
trees_gdf = gpd.read_file(tree_shp)

# Transform auf Raster-Index (row, col)
tree_coords = []
for x, y in zip(trees_gdf.geometry.x, trees_gdf.geometry.y):
    col, row = ~transform * (x, y)  # inverse transform
    col = int(round(col))
    row = int(round(row))
    # check bounds
    if 0 <= row < nrows and 0 <= col < ncols:
        tree_coords.append((col, row))

# numpy array wie von pycrown erwartet
tree_coords = np.array(tree_coords).T.astype(np.int32)  # shape (2, ntrees)
print("Number of tree tops:", tree_coords.shape[1])

In [ ]:
## Import CHM

In [ ]:
import rasterio
import numpy as np

# CHM Pfad
chm_path = "chm.tif"

with rasterio.open(chm_path) as src:
    chm = src.read(1)  # Single Band CHM
    chm = chm.astype(np.float32)  # sicherstellen float32
    transform = src.transform
    epsg = src.crs.to_epsg()
    nrows, ncols = chm.shape

print("CHM shape:", chm.shape)

## Execute dalponte crown delineation

In [ ]:
crowns = crown_dalponte_circ(
    chm_array,
    tree_coords,
    th_seed=0.7,
    th_crown=0.55,
    th_tree=15,
    max_crown=10
)

export_crowns_polygons(crowns)